In [ ]:
import numpy as np
import pandas as pd
import json
import codecs
import os
import sys
import datetime as dt
sys.path.append('../')

In [ ]:
import oci
from oci.config import validate_config
import multiprocessing as mp
import ocifs

In [ ]:
from dask.distributed import Client
from pandas import json_normalize
import dask.bag as db
import dask.dataframe as dd

In [ ]:
import multiprocessing as mp
mp.cpu_count()

In [ ]:
#namespace = "lrhwvlagph9z"
bucket_name = "tenen-raw-data"
config = oci.config.from_file(
        "~/.apikey/config",
        "DEFAULT")

In [ ]:
validate_config(config)

In [ ]:
object_storage_client = oci.object_storage.ObjectStorageClient(config)
namespace = object_storage_client.get_namespace().data
next_starts_with = None
object_list = object_storage_client.list_objects(namespace, bucket_name , fields='name, timeCreated, size', start=next_starts_with)
f = object_list.data.objects
namespace

In [ ]:
fs = ocifs.OCIFileSystem(config="~/.apikey/config", profile="DEFAULT")

In [ ]:
bucket_dir = {'oct' : "/oct/", 'nov': "/nov/", 'dec': "/dec/" }
target_file = {'oct' : 'tenen_high_lvl_oct.csv', 'nov': 'tenen_high_lvl_nov.csv', 'dec': 'tenen_high_lvl_dec.csv' }
products_file = {'oct' : 'tenen_products_oct.csv', 'nov': 'tenen_products_nov.csv', 'dec': 'tenen_products_dec.csv'}
hits_file = {'oct' : 'tenen_hits_oct.csv', 'nov': 'tenen_hits_nov.csv', 'dec': 'tenen_hits_dec.csv'}

In [ ]:
#obj_list = bucket_txt_file_list(object_storage_client, namespace, bucket_name, None, '.txt')
path2 = 'tenen-raw-data@' + namespace + bucket_dir['dec']
objlist = fs.glob(path2)
objlist

In [ ]:
client = Client(n_workers=mp.cpu_count(), memory_limit='8GB')

In [ ]:
def read_json_objects(path):
    jsonlines = []
    with fs.open(path , 'r') as f:
        jsonlines = f.readlines()
    return jsonlines

In [ ]:
def dask_bug_json_list(json_list, num_p):
    return db.from_sequence(json_list, npartitions=num_p).map(json.loads)

In [ ]:
def dask_bug_read_json_text(obj, nspace):
    path = 'tenen-raw-data@' + nspace + "/"
    storage_opt = {"config": "~/.apikey/config"}
    return db.read_text(path+obj, storage_options=storage_opt).map(json.loads)

In [ ]:
def general_json_to_df__1(dbug, **kwargs):
    ddf = dbug.to_dataframe()
    return ddf.compute()

In [ ]:
def general_json_to_df__2(dbug, **kwargs):
    extract_col = kwargs['col']
    ddf = (dbug.pluck(extract_col)).to_dataframe()
    df0 = ddf.compute()
    df0.reset_index(inplace=True)
    return df0.add_prefix(extract_col+'.')

In [ ]:
def parse_hits_column(x):
    df = pd.DataFrame()
    if(len(x['hits']) > 0):
        df = pd.json_normalize(x,
                               record_path='hits',
                               record_prefix='hits.',
                               meta=['fullVisitorId', 'visitStartTime']
                               )
    return df

In [ ]:
def parse_products_column(x):
    df = pd.DataFrame()
    if(len(x['hits']) > 0):
        df = pd.json_normalize(x,
                               record_path=['hits', 'product'],
                               record_prefix='hits.product.',
                               meta=['fullVisitorId', 'visitStartTime', ['hits', 'hitNumber']]
                               )
    return df

In [ ]:
def put_file(src, namespace, target):
    fs.put_file(src, 'oci://tenen-raw-data@'+namespace+target)

In [ ]:
def check_col_types(df):
    x = df.isna().any()
    x.to_csv("/home/datascience/files/isna.csv")
    return dict(df.dtypes)

In [ ]:
jsonobjs = []
for obj in objlist:
    jsonobjs = jsonobjs + read_json_objects(obj)
len(jsonobjs)

In [ ]:
dbug = dask_bug_json_list(jsonobjs, mp.cpu_count())

In [ ]:
cols =  ['totals', 'trafficSource', 'device', 'geoNetwork', 'privacyInfo']
df = general_json_to_df__1(dbug)
df.reset_index(inplace=True)
for c in cols:
    df_tmp =  general_json_to_df__2(dbug, col=c)
    df = pd.concat([df.drop(columns=[c]), df_tmp], axis=1)
col='trafficSource.adwordsClickInfo'
df0 = pd.json_normalize(df[col]).add_prefix(col+'.')
df = pd.concat([df.drop(columns=[col]), df0], axis=1)
df = df.drop(columns=['index', 'date', 'hits', 'customDimensions', 'totals.index', 'geoNetwork.index', 'trafficSource.index', 'device.index', 'privacyInfo.index'])
df['visitStartTime'] = pd.to_datetime(df['visitStartTime'], unit='s')
df.shape

In [ ]:
df = df.replace(["", "/", ":", "(not set)", "UNKNOWN"], "0").fillna("0").astype(str)

In [ ]:
df = df.dropna(how='all')
df.shape

In [ ]:
df.head(2)

In [ ]:
y = check_col_types(df)

In [ ]:
tfile = target_file['nov']
path = f"/home/datascience/files/{tfile}" 
#target = 'oci://tenen-raw-data@'+namespace+'/products/'+ target_file['nov']
#configf = {"config": "~/.apikey/config"}
df.to_csv(path, index=False, encoding='utf-8')

In [ ]:
dbl = db.map(lambda x: parse_hits_column(x), dbug).compute()
df0 = pd.concat(dbl)
df0.shape

In [ ]:
df0['numOfProducts'] = df0.apply(lambda x : len(x['hits.product']), axis=1)
df0['visitStartTime'] = pd.to_datetime(df0['visitStartTime'], unit='s')

In [ ]:
#df0['startTime'] = pd.to_datetime(df0['hits.hour'].astype(str) + ':' + df0['hits.minute'].astype(str), format='%H:%M').dt.time

In [ ]:
df0 = df0.drop(columns=['hits.product', 'hits.promotion', 'hits.experiment', 'hits.publisher_infos',
                        'hits.customVariables', 'hits.customDimensions', 'hits.customMetrics',
                        'hits.hour', 'hits.minute'])

In [ ]:
df0 = df0.replace(["", "/", ":", "(not set)", "UNKNOWN"], "0").fillna("0").astype(str)

In [ ]:
df0.shape

In [ ]:
df0.columns

In [ ]:
df0.tail(5)

In [ ]:
check_col_types(df0)

In [ ]:
tfile = hits_file['dec']
path = f"/home/datascience/files/{tfile}"
df0.to_csv(path, index=False, encoding='utf-8')

In [ ]:
dbp = db.map(lambda x: parse_products_column(x), dbug).compute()

In [ ]:
df1 = pd.concat(dbp)
df1['visitStartTime'] = pd.to_datetime(df1['visitStartTime'], unit='s')
df1 = df1.drop(columns=['hits.product.customMetrics'])
df1.shape

In [ ]:
df1.columns

In [ ]:
df1['hits.product.customDimensions'] = df1['hits.product.customDimensions'].apply(lambda y: [{'index':'0','value':'None'}] if len(y)==0 else y)

In [ ]:
df1 = df1.replace(["", "/", ":", "(not set)", "UNKNOWN"], "0").fillna("0").astype(str)
df1.head(3)

In [ ]:
tfile = products_file['dec']
path = f"/home/datascience/files/{tfile}"
df1.to_csv(path, index=False, encoding='utf-8')

In [ ]:
#fs.put_file(path+tfile, 'oci://tenen-raw-data@'+namespace+'/products/'+tfile)